In [1]:
import pandas as pd
import numpy as np
from IPython import display as ICD


from pickle import load,
from scripts.preprocess import load_index_data, load_tbills_data
from scripts.utils import print_results

In [53]:
def get_bootstrap_results(bootstrap):
    implied, params = {}, {}
    for n, res in enumerate(bootstrap.values()):
        start, end, pred = res.values()
        log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
        log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

        implied[n], params[n] = print_results(pred, log_mk, log_rf, False)
    
    implied = pd.concat(implied, axis=0)
    params = pd.concat(params, axis=0)
    sd = implied.groupby(level=1).std()
    
    implied = implied.groupby(level=1).mean()
    params = params.groupby(level=1).mean()
    
    implied.loc['std', ['alpha (%)', 'beta']] = sd.loc['value', ['alpha (%)', 'beta']].values
    return implied.reindex(index=['value', 'std']), params.reindex(index=['value', 'std'])

        
def gather_results(result_list):
    results = {}
    for name, path in result_list.items():
        res = load(open(path, 'rb'))
        start, end, pred = res.values()
        log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
        log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

        implied, params = print_results(pred, log_mk, log_rf, False)
        chunk = pd.concat([params, implied], axis=1)
        chunk.loc['value'] = chunk.loc['value'].apply(lambda x: f"{x:.2f}")
        chunk.loc['std'] = chunk.loc['std'].apply(lambda x: f"({x:.2f})")
        
        results[name] = chunk
    
    return pd.concat(results, axis=0)

# Full

## Boostrap

In [3]:
bootstrap = load(open('../../res_bootstrap_unpacked.pkl', 'rb'))
values = pd.DataFrame([sample.loc['value'] for sample in bootstrap.values()], index=range(len(bootstrap)))
boot_res = values.describe().loc[['mean', 'std']]

In [5]:
res = load(open('../../res.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo').iloc[:,0].values

implied, params = print_results(pred, log_mk, log_rf, False)
ICD.display(pred)
ICD.display(implied)
ICD.display(params)

,gamma,delta,sigma,k,a,b,pi_err
value,0.056164,-1.056373,0.195405,0.538005,0.522499,10.000180,0.276883
std,0.000289,0.002290,0.003481,0.005971,0.001495,0.000916,0.041095


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.952746,33.302074,32.291673,-1.131642
std,40.088604,42.146657,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.465588,-1.056373,39.081091,53.800463,0.522499,10.000180,27.688293
sd,0.115474,0.002290,0.696293,0.321248,0.000781,0.000916,0.822800


## Cochrane

In [32]:
res = load(open('./results/res_cochrane.pkl', 'rb'))
start, end = '1987-01-01', '2000-08-02'
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', test=True, prefix='../../').values

implied, params = print_results(res, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)


# Cochrane
# -2.12  0.96   99.23   16.04  0.76   4.76   1.26
# 1.09   0.09   1.75    0.79   0.02   0.13   0.9

Using parameters (annualized percentages)
E[log Rf]=5.26%, E[log Rm]=15.92%, V[log Rm]=14.80%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,34.308917,90.608776,71.6657,1.129408
std,97.328385,114.497715,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,18.808555,0.960933,96.283223,17.245886,0.748421,4.807627,5.765190
sd,1.054837,0.078594,2.167803,0.515302,0.021466,0.115496,0.572347


## Baseline

In [36]:
res = load(open('./results/res.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.45%, E[log Rm]=3.38%, V[log Rm]=8.46%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.935279,33.283870,32.292905,-1.131543
std,40.090256,42.147543,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.465588,-1.056373,39.081091,53.800463,0.522499,10.000180,27.688293
sd,0.115474,0.002290,0.696293,0.321248,0.000781,0.000916,0.822800


## Industries

In [37]:
res = load(open('./results/res_Health.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.45%, E[log Rm]=3.38%, V[log Rm]=8.46%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.996705,50.746309,50.138498,-1.426878
std,68.962810,75.437620,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.735532,-1.280739,68.106013,15.812377,0.571673,9.999444,9.937278
sd,1.201222,0.184304,3.330918,0.337230,0.004904,0.000812,2.878878


In [38]:
res = load(open('./results/res_Other.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.30%, E[log Rm]=3.31%, V[log Rm]=8.39%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,16.602002,19.292409,18.041192,-0.766489
std,21.164118,21.729294,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,15.052120,-0.739078,20.235602,86.947560,0.524608,9.992784,67.164687
sd,0.443088,0.049559,7.391515,1.063108,0.002278,0.000824,3.090092


In [39]:
res = load(open('./results/res_Retail.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.19%, E[log Rm]=3.36%, V[log Rm]=8.64%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,20.528945,29.067261,27.928585,-0.683958
std,38.809052,40.575767,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,19.087538,-0.644586,38.407755,59.136139,0.529450,9.986410,27.906953
sd,0.384297,0.067948,4.335142,1.597876,0.002101,0.000933,2.156463


In [40]:
res = load(open('./results/res_Tech.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=2.31%, E[log Rm]=3.36%, V[log Rm]=8.42%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,21.810904,29.105371,28.428608,-1.156679
std,35.452167,37.009672,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,20.648871,-1.090301,34.243466,61.838131,0.516370,9.998979,31.640361
sd,0.188914,0.020892,1.646535,0.339479,0.000994,0.000193,0.812557


## Before 2000

In [26]:
res = load(open('./results/res_2000.pkl', 'rb'))
start, end = '1987-01-01', '2000-08-02'
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(res, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=5.26%, E[log Rm]=5.42%, V[log Rm]=7.60%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,101.209273,529.746409,533.417333,-19.798031
std,217.333686,458.100872,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,97.325745,-8.700604,207.043480,9.641510,1.774183,2.101270,3.546142
sd,17.461552,0.597502,19.472328,3.485849,0.413910,0.259416,4.788300


In [27]:
res

,gamma,delta,sigma,k,a,b,pi_err
value,0.243314,-8.700604,1.035217,0.096415,1.774183,2.101270,0.035461
std,0.043654,0.597502,0.097362,0.361546,0.233296,0.259416,1.399928


# From 2010

## SP500

### Baseline

In [3]:
res = load(open('../results/res_from_2010.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,28.044147,38.597624,32.143804,1.369221
std,41.959509,46.519291,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.486135,1.263586,40.741357,58.001268,0.524208,9.976762,25.248838
std,0.561361,0.131019,1.632983,0.542034,0.001035,0.000852,0.753908


### Bootstrap

In [54]:
bootstrap = load(open('../results/res_from_2010_bootstrap.pkl', 'rb'))
implied, params = get_bootstrap_results(bootstrap)

ICD.display(implied)
ICD.display(params)

,E[ln R] (%),E[R] (%),alpha (%),beta
value,27.915278,38.943901,32.273601,1.418881
std,42.908761,47.645811,0.666767,0.101944


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.177828,1.308279,41.623125,56.730761,0.524225,9.975520,24.313234
std,0.399665,0.054684,1.939227,0.692920,0.001218,0.002243,0.990630


### Industries

In [ ]:
res = load(open('../results/res_from_2010_tech.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,30.447771,44.398815,36.028574,1.808849
std,48.281191,51.640742,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,23.350359,1.647008,46.474905,52.761381,0.535861,9.994694,22.433772
sd,0.573147,0.096091,2.414756,1.118730,0.001282,0.000661,1.117114


In [ ]:
res = load(open('../results/res_from_2010_other.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.049506,36.410122,33.125152,0.642284
std,48.575001,51.495145,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,20.171717,0.596017,48.343735,39.852130,0.493348,9.999882,14.341468
sd,0.410862,0.011073,2.297246,0.801886,0.001529,0.000870,1.007226


In [ ]:
res = load(open('../results/res_from_2010_health.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,24.738513,28.837675,24.074829,0.98131
std,24.928114,25.911634,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,20.534031,0.926460,23.817091,85.994521,0.545547,9.999827,54.834035
sd,1.589694,0.209106,17.892963,4.103842,0.005613,0.003894,5.782390


In [ ]:
res = load(open('../results/res_from_2010_retail.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,29.460272,38.650506,30.048132,1.862101
std,38.561903,40.760224,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,22.079368,1.717618,36.067626,63.138034,0.532096,9.996947,28.519121
sd,0.988565,0.081985,6.428220,1.994004,0.003466,0.000788,3.203769


## NASDAQ

### Baseline

In [ ]:
res = load(open('../results/res_from_2010_^IXIC.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,28.202291,38.678406,32.647855,1.272122
std,41.757693,44.210948,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,23.004577,1.173846,40.703365,58.106907,0.523157,9.994176,25.243450
sd,0.858080,0.011246,0.786116,0.572693,0.001619,0.003164,0.789164


### Industries

In [ ]:
res = load(open('../results/res_from_2010_tech_^IXIC.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,30.517223,40.637618,34.111902,1.385714
std,40.462530,42.906486,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,24.921910,1.272877,39.178912,63.761897,0.534729,9.999982,28.731251
sd,0.364498,0.056970,1.809385,0.569653,0.001500,0.000660,1.204127


In [ ]:
res = load(open('../results/res_from_2010_other_^IXIC.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.058567,36.491376,32.584336,0.784987
std,48.709469,51.646767,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,19.649959,0.728229,48.364774,39.894354,0.493309,9.999991,14.368795
sd,0.748723,0.154677,2.591675,1.089477,0.001475,0.000882,1.407250


In [ ]:
res = load(open('../results/res_from_2010_health_^IXIC.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,24.802254,28.768071,24.734164,0.814091
std,24.406496,25.363286,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,21.230728,0.768808,23.630132,86.440400,0.544876,9.999941,55.002857
sd,1.436257,0.197736,15.582735,3.498681,0.005244,0.000741,5.442236


In [ ]:
res = load(open('../results/res_from_2010_retail_^IXIC.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,29.846586,38.655690,31.500597,1.530093
std,37.559328,39.681683,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,23.693929,1.411696,35.846339,64.148843,0.533119,9.980365,28.557948
sd,1.408174,0.229937,8.413139,3.331919,0.003188,0.001673,4.078831


## Russel 2000

### Baseline

In [ ]:
res = load(open('../results/res_from_2010_^RUT.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,28.092102,38.556826,33.211427,1.114948
std,41.756048,44.203042,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,23.475060,1.029217,40.947903,56.874738,0.523091,9.998763,24.609609
sd,0.228733,0.036744,1.211804,0.291716,0.000894,0.000725,0.808638


### Industries

In [ ]:
res = load(open('../results/res_from_2010_tech_^RUT.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,30.920904,40.766758,35.077145,1.193911
std,39.748244,42.140039,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,26.033630,1.096524,38.782192,64.720869,0.536634,9.970507,28.958744
sd,0.380943,0.058135,1.995331,0.723783,0.001296,0.001796,1.183004


In [ ]:
res = load(open('../results/res_from_2010_other_^RUT.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,23.136821,36.507770,33.245029,0.637185
std,48.579482,51.505798,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,20.278551,0.591155,48.352004,39.878771,0.493396,10.000003,14.396481
sd,0.491275,0.081329,1.921303,0.491491,0.001602,0.000699,1.050168


In [ ]:
res = load(open('../results/res_from_2010_health_^RUT.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,25.240064,29.900777,26.197549,0.738233
std,26.817490,27.927192,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,21.963373,0.695372,26.242487,82.616020,0.546694,9.999519,52.414893
sd,1.259028,0.157604,7.127250,0.916394,0.005515,0.001235,6.027337


In [ ]:
res = load(open('../results/res_from_2010_retail_^RUT.pkl', 'rb'))
start, end, pred = res.values()
log_rf = load_tbills_data('TB3MS', start, end).iloc[:,0].values
log_mk = load_index_data('^SP500TR', start, end, '1mo', prefix='../../').values

implied, params = print_results(pred, log_mk, log_rf, True)
ICD.display(implied)
ICD.display(params)

Using parameters (annualized percentages)
E[log Rf]=0.48%, E[log Rm]=4.50%, V[log Rm]=7.94%


,E[ln R] (%),E[R] (%),alpha (%),beta
value,29.880628,39.421963,32.948448,1.373739
std,39.287685,41.578655,NaN,NaN


,gamma (%),delta,sigma (%),k (%),a,b,pi (%)
value,24.315431,1.265376,37.980183,59.757663,0.532558,9.997318,27.447623
sd,1.169309,0.159583,6.880552,2.717553,0.003813,0.000746,3.600361


# Aggregate

In [ ]:
results = gather_results({ 
    'SP500TR Baseline': '../results/res_from_2010.pkl',
    'SP500TR Tech': '../results/res_from_2010_tech.pkl',
    'SP500TR Retail': '../results/res_from_2010_retail.pkl', 
    'SP500TR Health': '../results/res_from_2010_health.pkl',
    'SP500TR Other': '../results/res_from_2010_other.pkl',
    'NASDAQ Baseline': '../results/res_from_2010_^IXIC.pkl',
    'NASDAQ Tech': '../results/res_from_2010_tech_^IXIC.pkl',
    'NASDAQ Retail': '../results/res_from_2010_retail_^IXIC.pkl', 
    'NASDAQ Health': '../results/res_from_2010_health_^IXIC.pkl',
    'NASDAQ Other': '../results/res_from_2010_other_^IXIC.pkl',
    'Russel Baseline': '../results/res_from_2010_^RUT.pkl',
    'Russel Tech': '../results/res_from_2010_tech_^RUT.pkl',
    'Russel Retail': '../results/res_from_2010_retail_^RUT.pkl', 
    'Russel Health': '../results/res_from_2010_health_^RUT.pkl',
    'Russel Other': '../results/res_from_2010_other_^RUT.pkl',})

results_sp500 = gather_results({ 
    'SP500TR Baseline': '../results/res_from_2010.pkl',
    'SP500TR Tech': '../results/res_from_2010_tech.pkl',
    'SP500TR Retail': '../results/res_from_2010_retail.pkl', 
    'SP500TR Health': '../results/res_from_2010_health.pkl',
    'SP500TR Other': '../results/res_from_2010_other.pkl',})

results_nasdaq = gather_results({ 
    'NASDAQ Baseline': '../results/res_from_2010_^IXIC.pkl',
    'NASDAQ Tech': '../results/res_from_2010_tech_^IXIC.pkl',
    'NASDAQ Retail': '../results/res_from_2010_retail_^IXIC.pkl', 
    'NASDAQ Health': '../results/res_from_2010_health_^IXIC.pkl',
    'NASDAQ Other': '../results/res_from_2010_other_^IXIC.pkl',})

results_russel = gather_results({
    'Russel Baseline': '../results/res_from_2010_^RUT.pkl',
    'Russel Tech': '../results/res_from_2010_tech_^RUT.pkl',
    'Russel Retail': '../results/res_from_2010_retail_^RUT.pkl', 
    'Russel Health': '../results/res_from_2010_health_^RUT.pkl',
    'Russel Other': '../results/res_from_2010_other_^RUT.pkl',})

In [ ]:
results

gamma (%)   delta sigma (%)   k (%)       a       b  \
SP500TR Baseline value     22.49    1.26     40.74   58.00    0.52    9.98   
                 std      (0.56)  (0.13)    (1.63)  (0.54)  (0.00)  (0.00)   
SP500TR Tech     value     23.35    1.65     46.47   52.76    0.54    9.99   
                 std      (0.57)  (0.10)    (2.41)  (1.12)  (0.00)  (0.00)   
SP500TR Retail   value     22.08    1.72     36.07   63.14    0.53   10.00   
                 std      (0.99)  (0.08)    (6.43)  (1.99)  (0.00)  (0.00)   
SP500TR Health   value     20.53    0.93     23.82   85.99    0.55   10.00   
                 std      (1.59)  (0.21)   (17.89)  (4.10)  (0.01)  (0.00)   
SP500TR Other    value     20.17    0.60     48.34   39.85    0.49   10.00   
                 std      (0.41)  (0.01)    (2.30)  (0.80)  (0.00)  (0.00)   
NASDAQ Baseline  value     23.00    1.17     40.70   58.11    0.52    9.99   
                 std      (0.86)  (0.01)    (0.79)  (0.57)  (0.00)  (0.00)   
NASDAQ Tech      value     24.92    1.27     39.18   63.76    0.53   10.00   
                 std      (0.36)  (0.06)    (1.81)  (0.57)  (0.00)  (0.00)   
NASDAQ Retail    value     23.69    1.41     35.85   64.15    0.53    9.98   
                 std      (1.41)  (0.23)    (8.41)  (3.33)  (0.00)  (0.00)   
NASDAQ Health    value     21.23    0.77     23.63   86.44    0.54   10.00   
                 std      (1.44)  (0.20)   (15.58)  (3.50)  (0.01)  (0.00)   
NASDAQ Other     value     19.65    0.73     48.36   39.89    0.49   10.00   
                 std      (0.75)  (0.15)    (2.59)  (1.09)  (0.00)  (0.00)   
Russel Baseline  value     23.48    1.03     40.95   56.87    0.52   10.00   
                 std      (0.23)  (0.04)    (1.21)  (0.29)  (0.00)  (0.00)   
Russel Tech      value     26.03    1.10     38.78   64.72    0.54    9.97   
                 std      (0.38)  (0.06)    (2.00)  (0.72)  (0.00)  (0.00)   
Russel Retail    value     24.32    1.27     37.98   59.76    0.53   10.00   
                 std      (1.17)  (0.16)    (6.88)  (2.72)  (0.00)  (0.00)   
Russel Health    value     21.96    0.70     26.24   82.62    0.55   10.00   
                 std      (1.26)  (0.16)    (7.13)  (0.92)  (0.01)  (0.00)   
Russel Other     value     20.28    0.59     48.35   39.88    0.49   10.00   
                 std      (0.49)  (0.08)    (1.92)  (0.49)  (0.00)  (0.00)   

                        pi (%) E[ln R] (%) E[R] (%) alpha (%)   beta  
SP500TR Baseline value   25.25       28.04    38.60     32.14   1.37  
                 std    (0.75)     (41.96)  (44.43)     (nan)  (nan)  
SP500TR Tech     value   22.43       30.45    44.40     36.03   1.81  
                 std    (1.12)     (48.28)  (51.64)     (nan)  (nan)  
SP500TR Retail   value   28.52       29.46    38.65     30.05   1.86  
                 std    (3.20)     (38.56)  (40.76)     (nan)  (nan)  
SP500TR Health   value   54.83       24.74    28.84     24.07   0.98  
                 std    (5.78)     (24.93)  (25.91)     (nan)  (nan)  
SP500TR Other    value   14.34       23.05    36.41     33.13   0.64  
                 std    (1.01)     (48.58)  (51.50)     (nan)  (nan)  
NASDAQ Baseline  value   25.24       28.20    38.68     32.65   1.27  
                 std    (0.79)     (41.76)  (44.21)     (nan)  (nan)  
NASDAQ Tech      value   28.73       30.52    40.64     34.11   1.39  
                 std    (1.20)     (40.46)  (42.91)     (nan)  (nan)  
NASDAQ Retail    value   28.56       29.85    38.66     31.50   1.53  
                 std    (4.08)     (37.56)  (39.68)     (nan)  (nan)  
NASDAQ Health    value   55.00       24.80    28.77     24.73   0.81  
                 std    (5.44)     (24.41)  (25.36)     (nan)  (nan)  
NASDAQ Other     value   14.37       23.06    36.49     32.58   0.78  
                 std    (1.41)     (48.71)  (51.65)     (nan)  (nan)  
Russel Baseline  value   24.61       28.09    38.56     33.21   1.11  
                 std    (0.81)     (41.76)